In [1]:
# Add locations of modules
push!(LOAD_PATH, "../src/")

4-element Array{String,1}:
 "@"      
 "@v#.#"  
 "@stdlib"
 "../src/"

In [5]:
# Required packages
using LinearAlgebra
using Convex, SCS, Mosek, MosekTools

using GraphicalModelLearning
using LightGraphs

# Function definitions
using Printf, SparseArrays
include("beta_scaling.jl")
include("active_learning.jl")

using DelimitedFiles

┌ Info: Precompiling GraphicalModelLearning [top-level]
└ @ Base loading.jl:1192


In [6]:
using Revise

In [7]:
import Random

### Experiments for Spin History Only case

#### Create the dictionary over samples

In [8]:
Random.seed!(10)

# Sample considering a simple factor graph
gm_true = FactorGraph([0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0])
#gm_true = FactorGraph([0.4 0.9 0.1; 0.9 0.4 0.1; 0.1 0.1 0.4])

┌ Info: assuming spin alphabet
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/models.jl:108


  (1, 2) => 

alphabet: spin
vars: 3
terms: 3


0.9
  (1, 3) => 0.1
  (2, 3) => 0.1


In [11]:
# Gibbs Samples
samples_T, samples_mixed = gibbs_sampling(gm_true, n_samples, T_regime())
print("done")

┌ Info: using Glauber dynamics v1 to generate T-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:328


done

In [8]:
samples_T

48×8 Array{Int64,2}:
 3384  3  -1  -1  -1  -1  -1   1
 7756  1   1   1   1   1   1   1
 5277  3   1   1   1   1   1   1
  934  2   1   1  -1   1  -1  -1
  936  1   1   1  -1  -1   1  -1
  142  2  -1   1  -1  -1   1  -1
  543  3   1  -1   1   1  -1   1
  945  1  -1  -1   1   1  -1   1
 4636  1  -1  -1   1  -1  -1   1
 3423  3  -1  -1   1  -1  -1  -1
 4763  2   1   1  -1   1   1  -1
  578  3   1  -1  -1   1  -1   1
  563  3  -1   1  -1  -1   1  -1
    ⋮                  ⋮        
  607  3  -1   1   1  -1   1   1
  195  1  -1   1  -1  -1   1  -1
  581  3  -1   1  -1  -1   1   1
  918  1  -1   1  -1   1   1  -1
 1060  2  -1   1  -1  -1  -1  -1
  979  2  -1   1   1  -1  -1   1
  578  3   1  -1   1   1  -1  -1
 4626  2  -1  -1   1  -1  -1   1
  982  1   1  -1   1  -1  -1   1
  190  2  -1   1   1  -1   1   1
  123  1  -1   1   1  -1   1   1
  128  2   1  -1   1   1  -1   1

In [9]:
size(samples_T)

(48, 8)

In [10]:
# Create a dictionary over samples_T
dict_samples_sho = Dict{UInt8, Array{Int64, 2}}()
num_spins = 3

# Node number is the key of the dictionary
# Should include those which are flips and those which are not
# For the cases where there are flips, we know node i is being updated because this is single-site dynamics and we see a change
# Where there are no flips, we don't know which node is being updated so include all possibilities 

# For each dict[key], array should be of form (n_samples, FLAG_flip, \sigma^{(t)}, \sigma^{(t+1)})

# First get all those samples where there is no flip
ind_samples_T_no_flip = []
ind_samples_T_flip = []

for ind_config = 1:size(samples_T,1)
    if dot(samples_T[ind_config,3:(2+num_spins)],samples_T[ind_config,(3+num_spins):end]) == num_spins
        push!(ind_samples_T_no_flip,ind_config)
    else
        push!(ind_samples_T_flip,ind_config)
    end
end

samples_T_flip = samples_T[ind_samples_T_flip,:]
samples_T_no_flip = samples_T[ind_samples_T_no_flip,:]

24×8 Array{Int64,2}:
 7756  1   1   1   1   1   1   1
 5277  3   1   1   1   1   1   1
  142  2  -1   1  -1  -1   1  -1
  543  3   1  -1   1   1  -1   1
 4636  1  -1  -1   1  -1  -1   1
 4763  2   1   1  -1   1   1  -1
  563  3  -1   1  -1  -1   1  -1
  140  1   1  -1  -1   1  -1  -1
 7537  2  -1  -1  -1  -1  -1  -1
 2263  3  -1  -1   1  -1  -1   1
  168  2   1  -1  -1   1  -1  -1
 7451  1  -1  -1  -1  -1  -1  -1
  209  1   1  -1   1   1  -1   1
 4866  1   1   1  -1   1   1  -1
 2278  3   1   1  -1   1   1  -1
  599  3   1  -1  -1   1  -1  -1
 7776  2   1   1   1   1   1   1
 5040  3  -1  -1  -1  -1  -1  -1
  607  3  -1   1   1  -1   1   1
  195  1  -1   1  -1  -1   1  -1
 4626  2  -1  -1   1  -1  -1   1
  190  2  -1   1   1  -1   1   1
  123  1  -1   1   1  -1   1   1
  128  2   1  -1   1   1  -1   1

In [11]:
display(samples_T_flip)

24×8 Array{Int64,2}:
 3384  3  -1  -1  -1  -1  -1   1
  934  2   1   1  -1   1  -1  -1
  936  1   1   1  -1  -1   1  -1
  945  1  -1  -1   1   1  -1   1
 3423  3  -1  -1   1  -1  -1  -1
  578  3   1  -1  -1   1  -1   1
 1087  1  -1   1   1   1   1   1
  973  2   1  -1  -1   1   1  -1
 3557  3   1   1  -1   1   1   1
  563  3  -1   1   1  -1   1  -1
  970  2   1   1   1   1  -1   1
 1060  2  -1  -1  -1  -1   1  -1
  977  2  -1  -1   1  -1   1   1
 1069  1  -1  -1  -1   1  -1  -1
 1071  1   1   1   1  -1   1   1
  933  2   1  -1   1   1   1   1
 1030  1   1  -1  -1  -1  -1  -1
 3535  3   1   1   1   1   1  -1
  581  3  -1   1  -1  -1   1   1
  918  1  -1   1  -1   1   1  -1
 1060  2  -1   1  -1  -1  -1  -1
  979  2  -1   1   1  -1  -1   1
  578  3   1  -1   1   1  -1  -1
  982  1   1  -1   1  -1  -1   1

In [12]:
display(samples_T_no_flip)

24×8 Array{Int64,2}:
 7756  1   1   1   1   1   1   1
 5277  3   1   1   1   1   1   1
  142  2  -1   1  -1  -1   1  -1
  543  3   1  -1   1   1  -1   1
 4636  1  -1  -1   1  -1  -1   1
 4763  2   1   1  -1   1   1  -1
  563  3  -1   1  -1  -1   1  -1
  140  1   1  -1  -1   1  -1  -1
 7537  2  -1  -1  -1  -1  -1  -1
 2263  3  -1  -1   1  -1  -1   1
  168  2   1  -1  -1   1  -1  -1
 7451  1  -1  -1  -1  -1  -1  -1
  209  1   1  -1   1   1  -1   1
 4866  1   1   1  -1   1   1  -1
 2278  3   1   1  -1   1   1  -1
  599  3   1  -1  -1   1  -1  -1
 7776  2   1   1   1   1   1   1
 5040  3  -1  -1  -1  -1  -1  -1
  607  3  -1   1   1  -1   1   1
  195  1  -1   1  -1  -1   1  -1
 4626  2  -1  -1   1  -1  -1   1
  190  2  -1   1   1  -1   1   1
  123  1  -1   1   1  -1   1   1
  128  2   1  -1   1   1  -1   1

In [16]:
samples_T_no_flip[:,2] .= 0
# Now create the dictionary
for current_spin = 1:num_spins
    samples_current_spin = samples_T_flip[findall(isequal(current_spin),samples_T_flip[:,2]),:]
    samples_current_spin[:,2] .= 1
    dict_samples_sho[current_spin] = vcat(samples_current_spin,samples_T_no_flip)
end
print("done")

done

In [17]:
dict_samples_sho

Dict{UInt8,Array{Int64,2}} with 3 entries:
  0x02 => [934 1 … -1 -1; 973 1 … 1 -1; … ; 123 0 … 1 1; 128 0 … -1 1]
  0x03 => [3384 1 … -1 1; 3423 1 … -1 -1; … ; 123 0 … 1 1; 128 0 … -1 1]
  0x01 => [936 1 … 1 -1; 945 1 … -1 1; … ; 123 0 … 1 1; 128 0 … -1 1]

In [18]:
dict_samples_sho[1]

32×8 Array{Int64,2}:
  936  1   1   1  -1  -1   1  -1
  945  1  -1  -1   1   1  -1   1
 1087  1  -1   1   1   1   1   1
 1069  1  -1  -1  -1   1  -1  -1
 1071  1   1   1   1  -1   1   1
 1030  1   1  -1  -1  -1  -1  -1
  918  1  -1   1  -1   1   1  -1
  982  1   1  -1   1  -1  -1   1
 7756  0   1   1   1   1   1   1
 5277  0   1   1   1   1   1   1
  142  0  -1   1  -1  -1   1  -1
  543  0   1  -1   1   1  -1   1
 4636  0  -1  -1   1  -1  -1   1
    ⋮                  ⋮        
  209  0   1  -1   1   1  -1   1
 4866  0   1   1  -1   1   1  -1
 2278  0   1   1  -1   1   1  -1
  599  0   1  -1  -1   1  -1  -1
 7776  0   1   1   1   1   1   1
 5040  0  -1  -1  -1  -1  -1  -1
  607  0  -1   1   1  -1   1   1
  195  0  -1   1  -1  -1   1  -1
 4626  0  -1  -1   1  -1  -1   1
  190  0  -1   1   1  -1   1   1
  123  0  -1   1   1  -1   1   1
  128  0   1  -1   1   1  -1   1

#### "Try" to learn the structure

In [22]:
using GraphicalModelLearning

In [23]:
# M-Regime Sampling
n_samples = 20000
samples_M, samples_mixed = gibbs_sampling(gm_true, n_samples, M_regime())

┌ Info: using Glauber dynamics v1 to generate M-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:403


([346 3 … -1 1; 708 1 … 1 1; … ; 94 1 … 1 1; 97 2 … -1 1], [3914 -1 -1 -1; 3453 -1 -1 1; … ; 1324 -1 1 1; 1382 1 -1 -1])

In [24]:
dict_samples_M_sho = get_sho_samples(samples_M)

Dict{Int64,Array{Int64,2}} with 3 entries:
  2 => [150 1 … -1 -1; 714 1 … 1 -1; … ; 94 0 … 1 1; 97 0 … -1 1]
  3 => [346 1 … -1 1; 541 1 … -1 -1; … ; 94 0 … 1 1; 97 0 … -1 1]
  1 => [131 1 … 1 -1; 121 1 … -1 1; … ; 94 0 … 1 1; 97 0 … -1 1]

In [28]:
learned_gm_RISE_flip = learn_glauber_dynamics_sho(dict_samples_M_sho, n_samples)

┌ Info: using JuMP for RISE to learn Glauber dynamics from SHO samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:587


3×3 Array{Float64,2}:
 0.0124695   0.870837    0.0812382 
 0.870837   -0.00261021  0.0933278 
 0.0812382   0.0933278   0.00443327

In [26]:
true_gm = [0.0 0.9 0.1; 0.9 0.0 0.1; 0.1 0.1 0.0]

3×3 Array{Float64,2}:
 0.0  0.9  0.1
 0.9  0.0  0.1
 0.1  0.1  0.0

In [29]:
learned_gm_RISE_sho = learn_glauber_dynamics_sho2(dict_samples_M_sho, n_samples)

┌ Info: using JuMP for RISE to learn Glauber dynamics from SHO samples v2
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:695


3×3 Array{Float64,2}:
 0.00733476   0.760801    0.0633369  
 0.760801    -0.00208234  0.0708959  
 0.0633369    0.0708959   0.000849266

In [30]:
learned_gm_RPLE_flip = learn_glauber_dynamics_sho(dict_samples_M_sho, n_samples, RPLE())

┌ Info: using JuMP for RPLE to learn Glauber dynamics from SHO samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:636


3×3 Array{Float64,2}:
 0.0115918   0.873391    0.0839716 
 0.873391   -0.00177943  0.0962509 
 0.0839716   0.0962509   0.00444651

In [35]:
learned_gm_RPLE_sho = learn_glauber_dynamics_sho2(dict_samples_M_sho, n_samples, RPLE())

┌ Info: using JuMP for RPLE to learn Glauber dynamics from SHO samples v2
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:752


3×3 Array{Float64,2}:
 0.00639941   0.750567    0.0565638 
 0.750567    -0.00166431  0.0647878 
 0.0565638    0.0647878   0.00102197

#### More complicated case

In [42]:
samples_T, samples_mixed = gibbs_sampling(gm_true, n_samples, T_regime())

┌ Info: using Glauber dynamics v1 to generate T-regime samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/sampling.jl:328


([686 3 … -1 1; 1540 1 … 1 1; … ; 20 1 … 1 1; 34 2 … -1 1], [5080 -1 -1 -1; 3456 -1 -1 1; … ; 665 -1 1 1; 700 1 -1 -1])

In [43]:
dict_samples_T_sho = get_sho_samples(samples_T)

Dict{Int64,Array{Int64,2}} with 3 entries:
  2 => [181 1 … -1 -1; 180 1 … 1 -1; … ; 20 0 … 1 1; 34 0 … -1 1]
  3 => [686 1 … -1 1; 690 1 … -1 -1; … ; 20 0 … 1 1; 34 0 … -1 1]
  1 => [205 1 … 1 -1; 218 1 … -1 1; … ; 20 0 … 1 1; 34 0 … -1 1]

In [44]:
learned_gm_RISE_T_flip = learn_glauber_dynamics_sho(dict_samples_T_sho, n_samples)

┌ Info: using JuMP for RISE to learn Glauber dynamics from SHO samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:587


3×3 Array{Float64,2}:
 -0.000278117   0.000102904  -8.13848e-8
  0.000102904  -0.000584112   0.00433094
 -8.13848e-8    0.00433094   -9.61294e-5

In [45]:
learned_gm_RISE_T_sho = learn_glauber_dynamics_sho2(dict_samples_T_sho, n_samples)

┌ Info: using JuMP for RISE to learn Glauber dynamics from SHO samples v2
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:695


3×3 Array{Float64,2}:
 0.00583255   0.218506    0.0481435 
 0.218506    -0.00268667  0.0307352 
 0.0481435    0.0307352   0.00280869

In [46]:
true_gm

3×3 Array{Float64,2}:
 0.0  0.9  0.1
 0.9  0.0  0.1
 0.1  0.1  0.0

In [47]:
learned_gm_RPLE_T_flip = learn_glauber_dynamics_sho(dict_samples_T_sho, n_samples, RPLE())

┌ Info: using JuMP for RPLE to learn Glauber dynamics from SHO samples
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:636


3×3 Array{Float64,2}:
 -0.000340842   3.27325e-6    0.0025095  
  3.27325e-6   -0.000723803   0.00245471 
  0.0025095     0.00245471   -0.000256115

In [48]:
learned_gm_RPLE_T_sho = learn_glauber_dynamics_sho2(dict_samples_T_sho, n_samples, RPLE())

┌ Info: using JuMP for RPLE to learn Glauber dynamics from SHO samples v2
└ @ GraphicalModelLearning /Users/gogol/Dropbox (Personal)/Research/LANL/Code/GML_Glauber_Dynamics.jl/src/GraphicalModelLearning.jl:752


3×3 Array{Float64,2}:
 0.0157311   0.681672    0.0847279 
 0.681672   -0.00688494  0.058279  
 0.0847279   0.058279    0.00455379